In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[21]:


import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
from torch import nn 
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F


# In[22]:


INPUT_DATA_DIR = '/scratch/napamegs/chunks_Ag3.1/'
TRAIN_DATA_COEFFICIENT = 0.75

files = []

for (dirpath, dirnames, filenames) in os.walk(INPUT_DATA_DIR):
    files.extend(filenames)
    break

train_files_finish = int(len(files) * TRAIN_DATA_COEFFICIENT)
train_files = files[0:train_files_finish]
test_files = files[train_files_finish:len(files)]


# In[23]:


class get_data(Dataset):
    
    def __init__(self, trainTest_file, root_dir):
        self.annotations = trainTest_file
        self.root_dir = root_dir
        
    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        
        full_path = os.path.join(self.root_dir, self.annotations[index])
        df = pd.read_csv(full_path)
        X = df.iloc[:,2:8].values.astype('float64')
        targets = df.iloc[:,8:].values.astype('float64')
        X,targets=torch.FloatTensor(X),torch.FloatTensor(targets)
        return X, targets


# In[24]:


batch_size = 10000

train_set = get_data(
    trainTest_file=train_files,
    root_dir="/scratch/napamegs/chunks_Ag3.1/"
)

test_set = get_data(
    trainTest_file=test_files,
    root_dir="/scratch/napamegs/chunks_Ag3.1/",
)

train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)


# In[25]:


class ResidualBlock(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, input_size)
    
    def forward(self, x):
        identity = x
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out += identity
        out = self.relu(out)
        return out

class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(6, 64)
        self.residual_block = ResidualBlock(64, 32)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 4)
        self.max_pool = nn.MaxPool1d(kernel_size=2, stride=2)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.max_pool(out)
        out = self.residual_block(out)
        out = self.fc2(out)
        out = self.max_pool(out)
        out = self.fc3(out)
        out = self.fc4(out)
        return out


# In[26]:


model = DNN()
optimizer = Adam(model.parameters(), lr=0.00001)
criterion = nn.MSELoss()

print(model)


# In[27]:


num_epochs = 100
for epoch in range(num_epochs):

    for batch_idx, (data, targets) in enumerate(train_loader):
        scores = model(data)
        print(scores,target)
        loss = criterion(scores, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        break

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")
    break

# In[ ]:





